In [ ]:
from convert_envy_to_reqtxt import convert_environment_yaml_to_requirements_txt
convert_environment_yaml_to_requirements_txt()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
print(requests)

<module 'requests' from 'd:\\CS\\projects\\nlp-image-captioning\\.conda\\lib\\site-packages\\requests\\__init__.py'>
